In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd 
from tqdm import tqdm
tqdm.pandas()
train_df = pd.read_csv("/kaggle/input/boundary-box-data/dataset/train.csv")
train_df_updated = train_df.copy()
def update_range(row):
    xmax = row['xmax']
    xmin = row['xmin']
    ymax = row['ymax']
    ymin = row['ymin']
    row['xmax'] = xmax*2
    row['xmin'] = xmin*2
    row['ymax'] = ymax*2
    row['ymin'] = ymin*2
    xmax = row['xmax']
    xmin = row['xmin']
    ymax = row['ymax']
    ymin = row['ymin']
    if (xmax>1920):
        row['xmax'] = 1920
    if (xmin<0):
        row['xmin'] = 0
    if (ymax>1080):
        row['ymax'] = 1080
    if (ymin<0):
        row['ymin'] = 0
    row['bbox'] = [row['xmin'],row['xmax'],row['ymin'],row['ymax']]
    return row
train_df_updated = train_df_updated.progress_apply(update_range,axis=1)
train_df2 = train_df_updated.copy()
train_df2 = train_df2.groupby(['image_path','class'])['bbox'].apply(list).reset_index(name='bboxes')
train_df2    

100%|██████████| 19950/19950 [00:16<00:00, 1230.00it/s]


,image_path,class,bboxes
0,000b0fd767c5ab2656b7ed8d43674777.jpg,3.0,"[[804.0, 984.0, 502.0, 648.0]]"
1,000ed1547634a24f09f22530065d46c9.jpg,7.0,"[[1020.0, 1388.0, 280.0, 426.0]]"
2,0011b4d3f9068a9efbb84dc3157b76c9.jpg,3.0,"[[1698.0, 1920, 446.0, 570.0], [0, 98.0, 396.0..."
3,0019a9ab6fd5ef3df915241bc323afb5.jpg,4.0,"[[0.0, 980.0, 186.0, 1080], [0, 648.0, 328.0, ..."
4,00349ffee00a2ce026ffb36e4e37caab.jpg,3.0,"[[1552.0, 1676.0, 360.0, 480.0], [1722.0, 1878..."
...,...,...,...
9463,ffdbbc2f9afb28c2741739f868c88670.jpg,7.0,"[[362.0, 482.0, 312.0, 418.0]]"
9464,ffe49f9e20bcefe4c0f6e8bc90643308.jpg,3.0,"[[270.0, 514.0, 596.0, 842.0]]"
9465,ffe49f9e20bcefe4c0f6e8bc90643308.jpg,7.0,"[[436.0, 606.0, 330.0, 486.0]]"
9466,fff53b56bb409098e9d1efd05ceba7f0.jpg,2.0,"[[1008.0, 1566.0, 526.0, 1028.0], [1358.0, 179..."


In [3]:
test_df = pd.read_csv("/kaggle/input/boundary-box-data/dataset/test.csv")
test_df.head()

,image_path
0,953ab1447c46ecfef67ab14629cd70c7.jpg
1,e4ddbaa7970fca225a51288ce5f7d3f9.jpg
2,5b8120d69607a077b7583334be3ba18b.jpg
3,138b1dc82005b4c33e4886260649d313.jpg
4,0f91ec1533b845b13089f8cf4e0a36f7.jpg


In [4]:
from sklearn.model_selection import train_test_split
df_train,df_valid = train_test_split(train_df2,test_size=0.1,random_state=42,shuffle=True)
df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

In [5]:
!mkdir /kaggle/working/Theme1_new_data/
!mkdir /kaggle/working/Theme1_new_data/images/
!mkdir /kaggle/working/Theme1_new_data/labels/
!mkdir /kaggle/working/Theme1_new_data/images/train/
!mkdir /kaggle/working/Theme1_new_data/images/validation/
!mkdir /kaggle/working/Theme1_new_data/labels/train/
!mkdir /kaggle/working/Theme1_new_data/labels/validation/
!mkdir /kaggle/working/Theme1_new_data/images/test/

In [6]:
output_path = "/kaggle/working/Theme1_new_data"
import os
os.listdir(output_path)

['images', 'labels']

In [7]:
df_train.head()

,image_path,class,bboxes
0,a1dd30a49538cfef88952ce1149c1f42.jpg,3.0,"[[582.0, 690.0, 450.0, 528.0], [522.0, 634.0, ..."
1,9b44562a4d05c9495f91df807196c38d.jpg,9.0,"[[0, 1100.0, 418.0, 954.0], [558.0, 1880.0, 43..."
2,c11965bb64ff33f64ed45997fcd48f3b.jpg,9.0,"[[0, 1164.0, 304.0, 556.0], [620.0, 1702.0, 30..."
3,1a7f5bd2246c0f16b7c6ba175f977cbc.jpg,3.0,"[[380.0, 528.0, 600.0, 674.0], [256.0, 432.0, ..."
4,2a9cb28710df69d75707c8cbce04a9b0.jpg,3.0,"[[374.0, 612.0, 362.0, 488.0], [522.0, 748.0, ..."


In [8]:
import shutil
import numpy as np
data_path = "/kaggle/input/boundary-box-data/dataset/images"
def process_data(data,data_type="train"):
    for i,row in tqdm(data.iterrows(),total=len(data)):
        image_name = row['image_path'][:-4]
        bounding_boxes = row['bboxes']
        classs = row['class']
        yolo_data=[]
        for bbox in bounding_boxes:
            xmin = bbox[0]
            xmax = bbox[1]
            ymin = bbox[2]
            ymax = bbox[3]
            width = (xmax-xmin)
            height = (ymax-ymin)
            x_center = xmin+width/2
            y_center = ymin+height/2
            x_center /= 1920.0
            y_center /= 1080.0
            width/=1920.0
            height/=1080.0
            yolo_data.append([classs,x_center,y_center,width,height])
        yolo_data = np.array(yolo_data)
        np.savetxt(os.path.join(output_path,f"labels/{data_type}/{image_name}.txt"),yolo_data,fmt=["%d","%f","%f","%f","%f"])
        shutil.copyfile(os.path.join(data_path,f"{image_name}.jpg"),os.path.join(output_path,f"images/{data_type}/{image_name}.jpg"))

In [9]:
process_data(df_train,data_type="train")

100%|██████████| 8521/8521 [01:26<00:00, 98.04it/s] 


In [10]:
process_data(df_valid,data_type="validation")

100%|██████████| 947/947 [00:08<00:00, 115.95it/s]


In [11]:
import shutil
import numpy as np
data_path = "/kaggle/input/boundary-box-data/dataset/images"
def process_test_data(data):
    for i,row in tqdm(data.iterrows(),total=len(data)):
        image_name = row['image_path'][:-4]
        shutil.copyfile(os.path.join(data_path,f"{image_name}.jpg"),os.path.join(output_path,f"images/test/{image_name}.jpg"))
process_test_data(test_df)

100%|██████████| 2092/2092 [00:23<00:00, 90.91it/s]


In [12]:
train_df.head()

,class,image_path,name,xmax,xmin,ymax,ymin
0,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,797.0,701.0,262.0,211.0
1,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,932.0,786.0,329.0,238.0
2,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,736.0,657.0,275.0,229.0
3,7.0,ea906a663da6321bcef78be4b7d1afff.jpg,BAD_BILLBOARD,986.0,786.0,136.0,0.0
4,8.0,1c7d48005a12d1b19261b8e71df7cafe.jpg,SAND_ON_ROAD,667.0,549.0,228.0,179.0


In [13]:
train_df['bbox'] = train_df['xmax'].astype(str) + ','+ train_df['xmin'].astype(str)+','+train_df['ymax'].astype(str) +','+train_df['ymin'].astype(str)

In [14]:
train_df.head()

,class,image_path,name,xmax,xmin,ymax,ymin,bbox
0,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,797.0,701.0,262.0,211.0,"797.0,701.0,262.0,211.0"
1,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,932.0,786.0,329.0,238.0,"932.0,786.0,329.0,238.0"
2,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,736.0,657.0,275.0,229.0,"736.0,657.0,275.0,229.0"
3,7.0,ea906a663da6321bcef78be4b7d1afff.jpg,BAD_BILLBOARD,986.0,786.0,136.0,0.0,"986.0,786.0,136.0,0.0"
4,8.0,1c7d48005a12d1b19261b8e71df7cafe.jpg,SAND_ON_ROAD,667.0,549.0,228.0,179.0,"667.0,549.0,228.0,179.0"


In [15]:
import ast

In [16]:
train_df.bbox = train_df.bbox.apply(ast.literal_eval)

In [17]:
#train_df = train_df.groupby("image_path")["bbox"].apply()

In [18]:
smart_yaml = { "train" : '/kaggle/working/Theme1_new_data/images/train',"val" : '/kaggle/working/Theme1_new_data/images/validation',"nc":11,"names": ["GRAFFITI","FADED SIGNAGE","POTHOLES","GARBAGE","CONSTRUCTION ROAD","BROKEN_SIGNAGE","BAD STREETLIGHT","BAD BILLBOARD","SAND ON ROAD","CLUTTER_SIDEWALK","UNKEPT_FACADE"]}

In [19]:
smart_yaml

{'train': '/kaggle/working/Theme1_new_data/images/train',
 'val': '/kaggle/working/Theme1_new_data/images/validation',
 'nc': 11,
 'names': ['GRAFFITI',
  'FADED SIGNAGE',
  'POTHOLES',
  'GARBAGE',
  'CONSTRUCTION ROAD',
  'BROKEN_SIGNAGE',
  'BAD STREETLIGHT',
  'BAD BILLBOARD',
  'SAND ON ROAD',
  'CLUTTER_SIDEWALK',
  'UNKEPT_FACADE']}

In [20]:
os.chdir('/kaggle/working')
!pwd

/kaggle/working


In [21]:
# import pickle as pkl
import yaml
#some_dict = {'a': 1, 'b': 2}

with open(r'smart_yaml.yaml', 'w') as file:
    doc = yaml.dump(smart_yaml,file)

# with open('output_file.pkl', 'rb') as fin:
#     new_dict = pkl.load(fin)

# new_dict

In [22]:
data = '/kaggle/working/Theme1_new_data'
data_img = '/kaggle/working/Theme1_new_data/images'
data_labels = '/kaggle/working/Theme1_new_data/labels'
data_img_train = '/kaggle/working/Theme1_new_data/images/train'
data_img_val = '/kaggle/working/Theme1_new_data/images/validation'
data_label_train = '/kaggle/working/Theme1_new_data/labels/train'
data_label_val  = '/kaggle/working/Theme1_new_data/labels/validation'

In [23]:
train_df.head(5)

,class,image_path,name,xmax,xmin,ymax,ymin,bbox
0,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,797.0,701.0,262.0,211.0,"(797.0, 701.0, 262.0, 211.0)"
1,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,932.0,786.0,329.0,238.0,"(932.0, 786.0, 329.0, 238.0)"
2,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,736.0,657.0,275.0,229.0,"(736.0, 657.0, 275.0, 229.0)"
3,7.0,ea906a663da6321bcef78be4b7d1afff.jpg,BAD_BILLBOARD,986.0,786.0,136.0,0.0,"(986.0, 786.0, 136.0, 0.0)"
4,8.0,1c7d48005a12d1b19261b8e71df7cafe.jpg,SAND_ON_ROAD,667.0,549.0,228.0,179.0,"(667.0, 549.0, 228.0, 179.0)"


In [24]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 14998, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 14998 (delta 0), reused 1 (delta 0), pack-reused 14995
Receiving objects: 100% (14998/14998), 14.03 MiB | 22.87 MiB/s, done.
Resolving deltas: 100% (10286/10286), done.


In [25]:
%cd yolov5

/kaggle/working/yolov5


In [26]:
yaml_dir = '/kaggle/working/smart_yaml.yaml'

In [63]:
# with open('/kaggle/working/output_file.pkl', 'rb') as fin:
#     new_dict = pkl.load(fin)
!python /kaggle/working/yolov5/train.py --img 640 --cfg /kaggle/working/yolov5/models/yolov5s.yaml  --batch 16 --epochs 15 --data /kaggle/working/smart_yaml.yaml --weights yolov5s.pt --workers 24 

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) ^C
Traceback (most recent call last):
  File "/kaggle/working/yolov5/train.py", line 55, in <module>
    from utils.loggers import Loggers
  File "/kaggle/working/yolov5/utils/loggers/__init__.py", line 29, in <module>
    wandb_login_success = wandb.login(timeout=30)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_login.py", line 76, in login
    configured = _login(**kwargs)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_login.py", line 297, in _login
    wlogin.prompt_api_key()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_login.py", line 220, in prompt_api_key
    key, status = self._prompt_api_key()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_login.py", line 204, in _prompt_api_key
    no_create=self._settings.force if self._settings else None,
  File "/opt/conda

In [ ]:
import torch

In [ ]:
os.listdir("runs/train/exp/weights")

In [ ]:
model_weights = 'runs/train/exp/weights/best.pt'

In [ ]:
#model.save(model_weights)

In [ ]:
test_paths = '/kaggle/input/smartathon-theme-1-with-yolov5-new/Theme1_new_data/images/test'

In [ ]:
# os.listdir(test_paths)

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt  --source /kaggle/input/smartathon-theme-1-with-yolov5-new/Theme1_new_data_FCNN/test --save-txt

In [ ]:
results = "/kaggle/working/yolov5/runs/detect/exp/labels/"
# with open(results, 'r') as f:
#     data = f.read()
# print(data)

In [ ]:
test_paths

In [ ]:
test_df

In [ ]:
print(5)

In [ ]:
diction = {"0":"GRAFFITI","1":"FADED SIGNAGE","2":"POTHOLES","3":"GARBAGE","4":"CONSTRUCTION ROAD","5":"BROKEN_SIGNAGE","6":"BAD STREETLIGHT","7":"BAD BILLBOARD","8":"SAND ON ROAD","9":"CLUTTER_SIDEWALK","10":"UNKEPT_FACADE" }

In [ ]:
len(os.listdir("/kaggle/working/yolov5/runs/detect/exp2/labels/"))

In [ ]:
test_df['image_path']

In [ ]:
ans = [["class", "image_path", "name", "xmax", "xmin", "ymax", "ymin"]]

In [ ]:
for dirname, _, filenames in os.walk("/kaggle/working/yolov5/runs/detect/exp2/labels/"):
    for filename in filenames:
        i = 0
        lst = []
        image_id = filename[:-4]+".jpg"
        with open(results+filename, 'r') as f:
            data = f.readline()
            data = data.split(" ")
            tst = (" ").join(data)
            tst = tst[:-1]
            xmax = int((float(data[1])+(float(data[3])/2))*1920)
            xmin = int((float(data[1])-(float(data[3])/2))*1920)
            ymax = int((float(data[2])+(float(data[4])/2))*1080)
            ymin = int((float(data[2])-(float(data[4])/2))*1080)
            ans.append([data[0], image_id, (diction[(tst[0])]),xmax,xmin,ymax, ymin])

%cat '../i/working/yolov5/runs/detect/exp/labels/results[0]'

In [ ]:
final = pd.DataFrame(ans[1:],columns=ans[0])
final.head()

In [ ]:
final

In [ ]:
final

In [ ]:
final['xmax'] = final['xmax']/2 ##it was given in the model....
final['xmin'] = final['xmin']/2
final['ymax'] = final['ymax']/2
final['ymin'] = final['ymin']/2

In [ ]:
#update

In [ ]:
final

In [ ]:
final.isnull().sum()

In [ ]:
final.to_csv('myans.csv',index =False)